In [2]:
import os, shutil
import keras

In [5]:
data_base_dir = "../data/raw"
x = os.listdir(data_base_dir + "/train")
y = os.listdir(data_base_dir + "/test1")
print(len(x))
print(len(y))

25000
12500
